# xcube server loading places from xcube geoDB

This notebook describes how to enable xcube server to load places from the [xcube geoDB](https://github.com/dcs4cop/xcube-geodb).

For this to work, the following requirements must be satisfied:

* xcube version must be >= 1.0.3
* The dedicated xcube plugin [xcube-geodb-places)](https://github.com/dcs4cop/xcube-geodb-places) must be installed:
  `cd ${projects}`
  `git clone https://github.com/dcs4cop/xcube-geodb-places.git`
  `cd xcube-geodb-places`
  `mamba activate xcube`
  `pip install -e .`
* access to the xcube geoDB (you need valid credntials)

## Configuration

To add feature data from a xcube geoDB collection, you must have access to the xcube geoDB. If you wish to use your own collection, make sure to add it first to the xcube geoDB. But you can also use one of the public collections, as shown in the example. 

You need to specify the collection you want to load from the xcube geoDB to the xcube server configuration, and (re-)start xcube server.

A sample configuration can be copied and adapted from [here](https://github.com/dcs4cop/xcube-geodb-places/blob/main/xcube_places_plugin/config.yml.example).

### Explanation of the configuration items

#### clientId and clientSecret:

`GEODB_AUTH_CLIENT_ID:      <your_client_id>`

`GEODB_AUTH_CLIENT_SECRET:  <your_client_secret>`

If you are using xcube geoDB via EDC, your clientId and clientSecret are stored as environment variables. You can retrieve them by starting a Jupyter Notebook on EDC, and issuing the commands:

```
import os

os.environ('GEODB_AUTH_CLIENT_ID')
os.environ('GEODB_AUTH_CLIENT_SECRET')
```


#### Identifier:

Some arbitrary name for the group of places. Can be anything, but must not contain spaces.


#### Title:

The title for the group of places, as it appears in the viewer.


#### Query:

A [PostGREST](https://postgrest.org/en/stable/references/api/tables_views.html)-compliant query to the geoDB. This query determines the places to load from the geoDB. The example (`collection-name?limit=100&depth=gt.10&select=list,of,columns`) translates to "fetch the columns `list`, `of`, and `columns` from the first 100 rows from the collection `collection-name`, where the value of column `depth` is greater than 10". There are plenty of other possibilities to design queries, which are described in the PostGREST documentation.


#### DatasetRefs:

The place groups defined in the configuration will only be shown for the datasets listed here.


#### PropertyMapping:

These fields control the display: you can define which column of the collection determines the label, the color, or the description for the places. For example, if you have a collection with a column `feature-name`, you may want to put here `label: feature-name`.

For further server configuration details, please refer to the documentation: https://xcube.readthedocs.io/en/latest/cli/xcube_serve.html#configuration-file

### Demonstration of a sample configuration

In [7]:
from xcube.webapi.viewer import Viewer

viewer = Viewer(server_config={
    "GeoDBConf": {
        "GEODB_API_SERVER_URL": "https://xcube-geodb.brockmann-consult.de",
        "GEODB_API_SERVER_PORT": 443,
        "GEODB_AUTH_AUD": "https://xcube-users.brockmann-consult.de/api/v2",
        "GEODB_AUTH_CLIENT_ID": "<your_client_id>",
        "GEODB_AUTH_CLIENT_SECRET": "<your_client_secret>",
        "PlaceGroups": [
            {
                "Identifier": "DE002L1_HAMBURG_UA2018",
                "Title": "Hamburg - Urban Atlas Wetlands",
                "Query": 'eea-urban-atlas_DE002L1_HAMBURG_UA2018?select=identifier&class_2018=eq.Wetlands',
                "PropertyMapping": {
                    "label": "identifier"
                },
                "DatasetRefs": ["hamburg-city-cube"]
            }
        ]
    },
    "DataStores": [
        {
            "Identifier": "demo",
            "StoreId": "s3",
            "StoreParams": {
                "root": "xcube-dcfs/edc-xc-viewer-data",
                "storage_options": {
                    "anon": True
                }
            },
            "Datasets": [
                {
                    "Identifier": "hamburg-city-cube",
                    "Path": "HH_CityCube_RGB.zarr"
                }
            ]
        }
    ],
   "Styles":[
       {
           "Identifier": "default",
           "ColorMappings": {
               "MCI":{
                   "ColorBar": "CMRmap",
                   "ValueRange": [0., 0.008]
               },
               "NDVI":{
                   "ColorBar": "RdYlGn",
                   "ValueRange": [-1., 1]
               },
               "B02":{
                   "ColorBar": "bone",
                   "ValueRange": [0., 0.25]
               },
               "B03":{
                   "ColorBar": "bone",
                   "ValueRange": [0., 0.25]
               },
               "B04":{
                   "ColorBar": "bone",
                   "ValueRange": [0., 0.25]
               },
               "B05":{
                   "ColorBar": "bone",
                   "ValueRange": [0., 0.25]
               },
               "B06":{
                   "ColorBar": "bone",
                   "ValueRange": [0., 0.25]
               },
               "B08":{
                   "ColorBar": "bone",
                   "ValueRange": [0., 0.25]
               },
               "rgb":{
                   "Red":{
                       "Variable": "B04",
                       "ValueRange": [0., 0.25]
                   },
                   "Green":{
                       "Variable": "B03",
                       "ValueRange": [0., 0.25]
                   },
                   "Blue":{
                       "Variable": "B02",
                       "ValueRange": [0., 0.25]
                   }
               }
           }
       }
   ]

})

### Start the viewer and inspect places; they have been loaded from the geoDB

For xcube viewer functionalities please checkout the documentation: https://xcube.readthedocs.io/en/latest/viewer.html#functionality

In [8]:
viewer.info()

Server: http://localhost:8002
Viewer: http://localhost:8002/viewer/?serverUrl=http://localhost:8002


In [9]:
viewer.show()


404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 4.52ms
